# ⚙️ Configuration Data & Functions ⚙️


In [ ]:
CONFIG_INSTANCE = "domo-community"
CONSOL_INSTANCE = "domo-community"

## ⚙️ Auth Objects


In [ ]:
import os
import domolibrary.client.DomoAuth as dmda

print(f"⚙️ retrieving config data from {CONFIG_INSTANCE} ⚙️")

## Enum only allows unique values, so config and consol are the same when both are headed to playstation-beta

config_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ.get("DOMO_DOJO_ACCESS_TOKEN"),
    domo_instance=CONFIG_INSTANCE,
)
consol_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ.get("DOMO_DOJO_ACCESS_TOKEN"),
    domo_instance=CONSOL_INSTANCE,
)

await config_auth.print_is_token("config")
await consol_auth.print_is_token("consol")

⚙️ retrieving config data from domo-community ⚙️
🎉 config token retrieved from domo-community ⚙️
🎉 consol token retrieved from domo-community ⚙️


True

## ⚙️ Company SQL


In [ ]:
import domolibrary.classes.DomoDataset as dmds

## companies to configure
COMPANY_SQL = "SELECT * FROM TABLE"
DATASET_ID = "8d2a8055-7918-4039-b67d-361647e96ea8"

monit_company_ls = await dmds.DomoDataset.query_dataset_private(
    auth=config_auth, dataset_id=DATASET_ID, sql=COMPANY_SQL
)

# Define companies to sync


In [ ]:
config_company_ls.head()

,domain,config_exception_pw
0,domo-community,0


## ⚙️ creds and consol datases


In [ ]:
import importlib
import datetime as dt

from enum import Enum

features_ds = dmds.DomoDataset(auth=consol_auth, id=env.CONSOL_FEATURES)

# ✔️ Execution Functions ✔️


## ✔️ generate day runs & update activity log ✔️


#### utility functions


In [ ]:
import datetime as dt
import pandas as pd
import math

from pprint import pprint
from Library.utils.chunk_execution import chunk_list


def generate_day_runs(start_date, end_date, chunk_days):
    """generates a list of start_dates x number of days apart"""

    if isinstance(start_date, str):
        "converting to datetime"
        start_date = dt.datetime.strptime(start_date, "%Y-%m-%d")

    if isinstance(end_date, str):
        end_date = dt.datetime.strptime(end_date, "%Y-%m-%d")

    if isinstance(end_date, dt.date):
        end_date = dt.datetime.combine(end_date, dt.datetime.min.time())

    num_days = (end_date - start_date).days

    day_runs = list(range(math.ceil(num_days / chunk_days)) or [0])

    return [
        {
            "start_date": start_date + dt.timedelta(days=chunk_days * i),
            "end_date": min(
                start_date + dt.timedelta(days=chunk_days * (i + 1)), dt.datetime.now()
            ),
        }
        for i in day_runs
    ]


def concat_list_dataframe(df_ls):
    df = None

    for elem in df_ls:
        if not isinstance(elem, pd.DataFrame) or len(elem.index) == 0:
            pass

        if df is None:
            df = elem
        else:
            df = pd.concat([df, elem], join="inner")
    return df

# Reset Datasets


In [ ]:
# # reset consol_datasets
# import httpx
# session = httpx.AsyncClient()


# await ConsolDataset.CONNECTORS.value.reset_dataset(
#     full_auth = consol_auth,
#     is_index = True,
#     debug = False )

# await session.aclose()

In [ ]:
# # reset consol_datasets

# import httpx
# session = httpx.AsyncClient()


# await ConsolDataset.DATASETS.value.reset_dataset(
#     full_auth = consol_auth,
#     is_index = True,
#     debug = False )

# await session.aclose()

In [ ]:
# import httpx
# session = httpx.AsyncClient()

# df = pd.read_csv ('csv/playstation-loyalty_datasets.csv')

# print(df.iloc[0])
# await ConsolDataset.DATASETS.value.upload_csv(upload_df = df,
#                                     upload_method = UPDATE_METHOD,
#                                     debug = True,
#                                     partition_key = 'playstation-loyalty',
#                                     is_index=True,
#                                     session = session )

# await session.aclose()

# Process One Instance Functions

## get_instance_activity_log


In [ ]:
import datetime as dt
import pandas as pd
import importlib
import httpx

import Library.DomoClasses.DomoAuth as dmda
import Library.DomoClasses.DomoActivityLog as dmal
import Library.DomoClasses.DomoDataset as dmds
import Library.utils.chunk_execution as ce

importlib.reload(dmal)


async def get_instance_activity_log(
    instance_auth,
    start_date,
    end_date,
    session: httpx.AsyncClient = None,
    debug: bool = False,
):
    import asyncio

    # process the data x days at a time.
    day_runs = generate_day_runs(
        start_date=start_date, end_date=end_date, chunk_days=CHUNK_DAYS
    )

    if debug:
        print(
            f"\n 🚀 retrieving data from {instance_auth.domo_instance} from {start_date} -- there are {len(day_runs)} - date chunks"
        )

    instance_activity_log_df_list = await asyncio.gather(
        *[
            dmal.DomoActivityLog.get_activity_log(
                full_auth=instance_auth,
                end_time=day_run.get("end_date"),
                start_time=day_run.get("start_date"),
                session=session,
                debug=False,
            )
            for day_run in day_runs
        ]
    )
    if debug:
        print(
            f"get activity log: {type(instance_activity_log_df_list)} for {instance_auth.domo_instance}"
        )

    return concat_list_dataframe(instance_activity_log_df_list)

## get_instance_allowlist


In [ ]:
import Library.Automation.automation as au


async def get_instance_allowlist(
    instance_auth, session: httpx.AsyncClient, debug: bool = False
) -> pd.DataFrame:
    """return a dataframe data in the correct shape for upload for ONE instance"""
    try:
        df = await au.get_instance_whitelist_df(
            instance_auth=instance_auth, session=session
        )

        return df

    except Exception as e:
        print(f"getting data : unexpected error: {e}")
        return None

## get_instance_sandbox


In [ ]:
import datetime as dt
import pandas as pd
import importlib
import httpx

import Library.DomoClasses.DomoAuth as dmda
import Library.DomoClasses.DomoSandbox as dmsb
import Library.utils.chunk_execution as ce

importlib.reload(dmsb)


async def get_repo_lineage_ls(repo: dmsb.DomoRepository):
    await repo.lineage.get()
    df = repo.convert_lineage_to_dataframe(return_raw=False)
    df["instance"] = repo.full_auth.domo_instance
    df["report_date"] = dt.datetime.now()

    return df


async def get_instance_sandbox(
    instance_auth,
    session: httpx.AsyncClient = None,
    debug_prn: bool = False,
    debug: bool = False,
):
    import asyncio

    if debug_prn:
        print(f"\n 🚀 retrieving data from {instance_auth.domo_instance}")

    instance_repo_ls = await dmsb.DomoSandbox.get_repositories(instance_auth, debug)

    if not instance_repo_ls:
        print(f"\n 🚀 {instance_auth.domo_instance} has no repos")
        return None

    lineage_df_ls = await asyncio.gather(
        *[get_repo_lineage_ls(repo) for repo in instance_repo_ls]
    )

    return concat_list_dataframe(lineage_df_ls)

## Instance_Datasets


In [ ]:
import httpx
import datetime as dt

from Library.DomoClasses import DomoDatacenter as dmdc
from Library.DomoClasses import DomoDataset as dcds
from Library.DomoClasses import DomoCertification as dmcert
from Library.DomoClasses import DomoStream as dcdstr

from Library.Integrations import MonitDataset as igmd

importlib.reload(dcdstr)
importlib.reload(dmdc)
importlib.reload(dcds)


def process_dataset(ds, stream_ds=None, partition_ls=None, debug_prn: bool = False):
    """return a dataframe data in the correct shape for upload for ONE instance"""

    if debug_prn:
        print(f"🚧 starting {ds.id} - {ds.name} in {full_auth.domo_instance}")

    obj = igmd.MonitDataset_Record(
        id=ds.id,
        name=ds.name,
        data_provider_type=ds.data_provider_type,
        display_type=ds.display_type,
        stream_id=int(ds.stream_id) if ds.stream_id else None,
        instance=ds.full_auth.domo_instance,
        owner=ds.owner.name,
        row_count=int(ds.row_count) if ds.row_count else None,
        url=ds.display_url(),
        report_date=dt.datetime.now(),
    )

    if ds.tags.tag_ls:
        # print(ds.tags.tag_ls)
        obj.tags = ", ".join(ds.tags.tag_ls)

    if ds.certification:
        obj.is_certified = (
            (
                1
                if ds.certification.certification_state
                == dmcert.DomoCertificationState.CERTIFIED.value
                else 0
            ),
        )
        obj.certification_name = getattr(ds.certification, "certification_name")

    if stream_ds is not None:
        obj.stream_account_id = (
            int(stream_ds.account_id) if stream_ds.account_id else None
        )
        obj.stream_account_userid = (
            int(stream_ds.account_userid) if stream_ds.account_userid else None
        )
        obj.stream_config_tables = ",".join(stream_ds.configuration_tables)
        obj.stream_config_query = stream_ds.configuration_query or ""
        obj.update_method = stream_ds.update_method

        obj.connector_version = (
            float(stream_ds.transport_version) if stream_ds.transport_version else None
        )
        obj.connector_id = stream_ds.transport_description

    if partition_ls is not None:
        obj.nb_partitions = len(partition_ls)

    return obj


async def get_stream(ds, session, instance_auth, debug: bool = False):
    try:
        if debug:
            print(
                f"get_stream: retrieving {ds.stream_id} for {ds.id}"
                in {instance_auth.domo_instance}
            )

        return await dcdstr.DomoStream.get_definition(
            stream_id=ds.stream_id, full_auth=instance_auth, session=session
        )
    except:
        print(
            f"get_stream: error retrieving {ds.stream_id} for {ds.id} in {instance_auth.domo_instance}"
        )
        return None


async def get_partitions(ds, instance_auth, session):
    try:
        return await ds.list_partitions(full_auth=instance_auth, session=session)
    except:
        print(
            f"error retrieving partitions for {ds.id} in {instance_auth.domo_instance}"
        )
        return None


async def get_monitdataset_data(
    ds, session, instance_auth, debug: bool = False, debug_prn: bool = False
):

    stream = None
    partition_ls = None

    if debug_prn:
        print(
            f"get_monit_data:  retrieving augmentation for {ds.id} in {instance_auth.domo_instance}"
        )

    try:
        stream_ds = await get_stream(
            ds=ds, session=session, instance_auth=instance_auth, debug=debug
        )

    except:
        print(
            f"🆘 get_stream: error retrieving stream id {ds.stream_id} for {ds.id} in {instance_auth.domo_instance}"
        )

    try:
        partition_ls = await get_partitions(
            ds=ds, session=session, instance_auth=instance_auth
        )
    except:
        partition_ls = None

    if debug_prn:
        print(
            f"done retrieving augmentation for {ds.id} in {instance_auth.domo_instance}"
        )

    return igmd.MonitDataset_Data(ds=ds, stream_ds=stream_ds, partition_ls=partition_ls)

In [ ]:
async def get_instance_datasets(
    instance_auth,
    session: httpx.AsyncClient = None,
    debug: bool = False,
    debug_prn: bool = True,
) -> pd.DataFrame:
    import pandas as pd
    import asyncio

    # try:
    is_close_session = True if not session else False

    session = session or httpx.AsyncClient()

    if debug_prn:
        print(f"🎬 searching datacenter for {instance_auth.domo_instance}")

    instance_datasets = await dmdc.DomoDatacenter.search_datasets(
        full_auth=instance_auth, debug=debug, session=session
    )

    if debug_prn:
        print(
            f"🎉 get_instance_datasets: {len(instance_datasets)} dataset returned from {instance_auth.domo_instance}"
        )

        # except:
        print(
            f"🆘 get_instance_datasets: error retrieving datasets from {instance_auth.domo_instance}"
        )

    # try:
    monit_data = await asyncio.gather(
        *[
            get_monitdataset_data(ds, session=session, instance_auth=instance_auth)
            for index, ds in enumerate(instance_datasets)
        ]
    )

    if debug_prn:
        print(
            f"🎉🎉 get_instance_datasets: retrieved {len(monit_data)} from {instance_auth.domo_instance} -- starting augmentation"
        )

    monit_records = [
        process_dataset(
            ds=row.ds, stream_ds=row.stream_ds, partition_ls=row.partition_ls
        )
        for row in monit_data
    ]

    if is_close_session:
        await session.aclose()

    return pd.DataFrame(monit_records)


#     # except:
#         print(f"🆘 error augmenting datasets from {instance_auth.domo_instance}" )

#     finally:
#         await session.aclose()

In [ ]:
## test
ds_id = "c888d07d-5174-4340-b14a-6b6b50be0d15"
instance = config_company_ls[
    config_company_ls["domo_instance"] == "playstation-beta"
].iloc[0]
print(instance)
session = httpx.AsyncClient()
ds = await get_instance_datasets(
    instance_auth=instance["instance_auth"], session=session, debug_prn=True
)
await session.aclose()

display(HTML(ds.head().to_html()))

instance_created_date          2020-02-07
domo_instance            playstation-beta
project                             other
config_useprod                          1
Name: 2, dtype: object


AttributeError: 'str' object has no attribute 'domo_instance'

## Instance_Connectors


In [ ]:
import httpx

from Library.DomoClasses import DomoInstanceConfig as dmdic

importlib.reload(dmdic)


async def get_instance_connectors(
    instance_auth, session: httpx.AsyncClient, debug: bool = False
):
    """return a dataframe data in the correct shape for upload for ONE instance"""
    try:
        instance_connectors = await dmdic.DomoInstanceConfig.get_connectors(
            full_auth=instance_auth, debug=debug, session=session
        )

        for index, conn in enumerate(instance_connectors):
            if conn.tags:
                conn.tags = ", ".join(conn.tags)

            if conn.capabilities:
                conn.capabilities = ", ".join(conn.capabilities)

            instance_connectors[index] = conn

        upload_df = pd.DataFrame(instance_connectors)
        upload_df["instance"] = instance_auth.domo_instance

        return upload_df

    except Exception as e:
        print(f"getting data : unexpected error: {e}")
        return None


# await get_instance_connectors(instance = config_company_ls.iloc[0] )

## Instance_Features


In [ ]:
import Library.DomoClasses.DomoBootstrap as dmbsr

importlib.reload(dmbsr)


async def get_instance_features(
    instance_auth, session: httpx.AsyncClient, debug: bool = False
) -> pd.DataFrame:
    """return a dataframe data in the correct shape for upload for ONE instance"""
    try:
        instance_features = await dmbsr.DomoBootstrap.get_features(
            full_auth=instance_auth, debug=debug, session=session
        )

        upload_df = pd.DataFrame(instance_features)
        upload_df["instance"] = instance_auth.domo_instance

        return upload_df

    except Exception as e:
        print(f"getting data : unexpected error: {e}")
        return None


# await get_instance_bootstrap(instance = config_company_ls.iloc[0] )

In [ ]:
import httpx
import pandas as pd
import importlib

import Library.DomoClasses.DomoInstanceConfig as dmdic

importlib.reload(dmdic)


async def get_instance_grants(
    instance_auth,
    session: httpx.AsyncClient = None,
    debug: bool = False,
    debug_prn: bool = True,
):
    try:
        session = session or httpx.AsyncClient()

        if debug_prn:
            print(f"searching datacenter for {instance_auth.domo_instance}")

        instance_grants = await dmdic.DomoInstanceConfig.get_grants(
            full_auth=instance_auth, debug=debug, session=session
        )

        if debug_prn:
            print(
                f"get_instance_grants: {len(instance_grants)} grants returned from {instance_auth.domo_instance}"
            )

        await session.aclose()

        df = pd.DataFrame(instance_grants)
        df.depends_on = df.depends_on.apply(lambda x: ",".join(x))
        df.role_membership = df.role_membership.apply(
            lambda x: ",".join(x) if len(x) > 0 else None
        )
        df["instance"] = instance_auth.domo_instance

        return df

    except:
        print(
            f"🆘 get_instance_grants: error retrieving grants from {instance_auth.domo_instance}"
        )

    finally:
        await session.aclose()

## get Custom Roles


In [ ]:
import Library.DomoClasses.DomoInstanceConfig as dmic

importlib.reload(dmic)


async def process_role(role):
    await role.get_membership()
    return role


async def get_instance_roles(
    instance_auth, session: httpx.AsyncClient, debug: bool = False
) -> pd.DataFrame:
    """return a dataframe data in the correct shape for upload for ONE instance"""
    try:
        roles = await dmic.DomoInstanceConfig.get_roles(
            full_auth=instance_auth, debug=debug, session=session
        )

        roles_clean = await asyncio.gather(*[process_role(role) for role in roles])

        upload_df = pd.DataFrame(roles_clean)

        return upload_df

    except Exception as e:
        print(f"getting data : unexpected error: {e}")
        return None


# await get_instance_bootstrap(instance = config_company_ls.iloc[0] )

## get allowed domains


In [ ]:
import Library.DomoClasses.DomoInstanceConfig as dmic

importlib.reload(dmic)
import datetime as dt


async def get_instance_domains(
    instance_auth,
    session: httpx.AsyncClient = httpx.AsyncClient(),
    report_date: dt.datetime = dt.date.today(),
    debug: bool = False,
) -> pd.DataFrame:
    """return a dataframe data in the correct shape for upload for ONE instance"""
    # try:
    domains = await dmic.DomoInstanceConfig.get_authorized_domains(
        full_auth=instance_auth, debug=debug, session=session
    )

    # print(type(domains))

    upload_df = pd.DataFrame(domains, columns=["domain"])
    upload_df["instance"] = instance_auth.domo_instance
    upload_df["timestamp"] = report_date

    return pd.DataFrame(upload_df)


#     except Exception as e:
#         print(f"getting data : unexpected error: {e}")
#         return None

# config_company_ls.iloc[0]
# await get_instance_domains(instance_auth = config_company_ls.iloc[0]['full_auth'],  )

## get custom Groups


In [ ]:
import Library.DomoClasses.DomoGroup as dmgr

importlib.reload(dmgr)


async def get_instance_groups(
    instance_auth, session: httpx.AsyncClient, debug: bool = False
) -> pd.DataFrame:
    """return a dataframe data in the correct shape for upload for ONE instance"""
    try:
        groups = await dmgr.DomoGroups.all_groups(
            full_auth=instance_auth, debug=debug, session=session
        )

        upload_df = pd.DataFrame(
            [{"name": group.name, "type": group.type} for group in groups]
        )

        return upload_df

    except Exception as e:
        print(f"getting data : unexpected error: {e}")
        return []


# await get_instance_bootstrap(instance = config_company_ls.iloc[0] )

## get allowlist


In [ ]:
import Library.DomoClasses.DomoInstanceConfig as dmic

importlib.reload(dmic)


async def get_instance_allowList(
    instance_auth, session: httpx.AsyncClient, debug: bool = False
) -> pd.DataFrame:
    """return a dataframe data in the correct shape for upload for ONE instance"""
    # try:
    allowlist = await dmic.DomoInstanceConfig.get_whitelist(
        full_auth=instance_auth, debug=debug, session=session
    )

    upload_df = pd.DataFrame(allowlist)

    return upload_df

# Process Instance

## Get_Config


In [ ]:
import datetime as dt
import asyncio
import httpx


async def get_config(
    instance_auth: dmda.DomoFullAuth,
    instance_created_date,
    debug: bool = False,
    report_date: dt.date = dt.date.today(),
):

    instance_session = httpx.AsyncClient()

    try:
        groups_df, roles_df, domains_df, allowlist_df = await asyncio.gather(
            get_instance_groups(
                instance_auth=instance_auth, debug=debug, session=instance_session
            ),
            get_instance_roles(
                instance_auth=instance_auth, debug=debug, session=instance_session
            ),
            get_instance_domains(
                instance_auth=instance_auth, debug=debug, session=instance_session
            ),
            get_instance_allowList(
                instance_auth=instance_auth, debug=debug, session=instance_session
            ),
        )

        log = {
            "instance": instance_auth.domo_instance,
            "creation_date": instance_created_date,
            "url": f"{instance_auth.domo_instance}.domo.com",
            "role_admin_membership": [],
            "role_sie_admin_membership": [],
            "roles": [],
            "report_date": report_date,
        }

        log["groups"] = [group["name"] for index, group in groups_df.iterrows()]

        for index, role in roles_df.iterrows():
            if role["is_system_role"] == 0:
                log["roles"].append(role["name"])

            if role["name"] == "Admin":
                for member in role["membership_list"]:
                    log["role_admin_membership"].append(member.get("email"))

            if role["name"] == "sie_instance_admin":
                for member in role["membership_list"]:
                    log["role_sie_admin_membership"].append(member.get("email"))

        log["domains"] = [domain["domain"] for index, domain in domains_df.iterrows()]

        log["allowlist"] = [ip[0] for index, ip in allowlist_df.iterrows()]

        return log

    finally:
        await instance_session.aclose()


await get_config(
    instance_auth=config_company_ls.iloc[0]["instance_auth"],
    instance_created_date="2022-9-12",
)

{'instance': 'playstation',
 'creation_date': '2022-9-12',
 'url': 'playstation.domo.com',
 'role_admin_membership': ['JaeMyong.Wilson@sony.com',
  'john.taft@sony.com',
  'nikky.gangani@domo.com',
  'oleksii.zakrevskyi1@sony.com',
  'OmprakashBasant.Gurubaxani@sony.com',
  'sie.domo.admin@sony.com',
  'thomas.murphey@sony.com',
  'William.Honaker@sony.com'],
 'role_sie_admin_membership': [],
 'roles': ['sie_instance_admin',
  'admin_reset_account',
  'sie_participant_user',
  'sie_priviliged_user'],
 'report_date': datetime.date(2023, 1, 13),
 'groups': ['BDO - Restricted', 'Content', 'Default', 'PXG05 Accounts'],
 'domains': ['sony.com', 'domo.com'],
 'allowlist': ['10.84.32.0/20',
  '10.85.96.0/20',
  '103.123.136.0/22',
  '103.123.139.16/28',
  '103.123.139.32/30',
  '103.123.139.36/32',
  '103.123.139.5/32',
  '103.123.139.6/31',
  '103.123.139.8/29',
  '107.21.98.1',
  '13.92.125.193',
  '136.36.46.29',
  '160.33.110.18',
  '160.33.112.0/20',
  '160.33.240.0/20',
  '172.26.0.0/16

## Upload_Data


# Main Function


In [ ]:
# for dataset in ConsolDataset:
#     await dataset.value.reset_dataset()

# await ConsolDataset.ACTIVITY_LOG.value.reset_dataset()

In [ ]:
async def index_datasets():

    print(f"🥫🥫 indexing datasets")
    await asyncio.gather(*[ds.value.index_dataset() for ds in ConsolDataset])
    print(f"🥫🥫 indexing done")

In [ ]:
import asyncio
import pandas as pd
import time

import Library.utils.chunk_execution as ce
import Library.utils.upload_data as upd

importlib.reload(ce)


async def main(debug: bool = False, print_progress: bool = False):
    # try:
    script_start = time.time()

    await consol_auth.get_auth_token()
    if not consol_auth.token:
        print(f"invalid consolidation credentials to {consol_auth.domo_instance}")
        return

    cn_list = ce.chunk_list(
        config_company_ls[config_company_ls["is_valid"] == 1], CHUNK_INSTANCE
    )
    # print(cn_list)

    for cn_index, df in enumerate(cn_list):
        print(f"starting {cn_index}")

        allowlist_res = await asyncio.gather(
            *[
                upd.upload_data(
                    instance_auth=company.instance_auth,
                    consol_auth=consol_auth,
                    data_fn=get_instance_allowlist,
                    consol_ds=ConsolDataset.ALLOW_LIST.value,
                )
                for index, company in df.iterrows()
            ]
        )

        grants_res = await asyncio.gather(
            *[
                upd.upload_data(
                    instance_auth=company.instance_auth,
                    consol_auth=consol_auth,
                    data_fn=get_instance_grants,
                    consol_ds=ConsolDataset.GRANTS.value,
                )
                for index, company in df.iterrows()
            ]
        )

        allowdomain_res = await asyncio.gather(
            *[
                upd.upload_data(
                    instance_auth=company.instance_auth,
                    consol_auth=consol_auth,
                    data_fn=get_instance_domains,
                    consol_ds=ConsolDataset.ALLOW_DOMAINS.value,
                )
                for index, company in df.iterrows()
            ]
        )

        datasets_res = await asyncio.gather(
            *[
                upd.upload_data(
                    instance_auth=company.instance_auth,
                    consol_auth=consol_auth,
                    data_fn=get_instance_datasets,
                    consol_ds=ConsolDataset.DATASETS.value,
                    is_index=True,
                )
                for index, company in df.iterrows()
                # if company.full_auth.domo_instance == 'playstation-operations-infoteam'
            ]
        )

        connectors_res = await asyncio.gather(
            *[
                upd.upload_data(
                    instance_auth=company.instance_auth,
                    consol_auth=consol_auth,
                    data_fn=get_instance_connectors,
                    is_index=True,
                    consol_ds=ConsolDataset.CONNECTORS.value,
                )
                for index, company in df.iterrows()
            ]
        )

        features_res = await asyncio.gather(
            *[
                upd.upload_data(
                    instance_auth=company.instance_auth,
                    consol_auth=consol_auth,
                    data_fn=get_instance_features,
                    consol_ds=ConsolDataset.FEATURES.value,
                )
                for index, company in df.iterrows()
            ]
        )

        activitylog_res = await asyncio.gather(
            *[
                upd.upload_data_with_date(
                    instance_auth=company.instance_auth,
                    consol_auth=consol_auth,
                    data_fn=get_instance_activity_log,
                    consol_ds=ConsolDataset.ACTIVITY_LOG.value,
                    partition_date_col="date",
                    partition_delimiter=PARTITION_DELIMITER,
                    start_date=max(START_DATE, company["instance_created_date"]),
                    end_date=END_DATE,
                    debug=debug,
                )
                for index, company in df.iterrows()
            ]
        )

        sandboxlog_res = await asyncio.gather(
            *[
                upd.upload_data(
                    instance_auth=company.instance_auth,
                    consol_auth=consol_auth,
                    data_fn=get_instance_sandbox,
                    consol_ds=ConsolDataset.SANDBOX.value,
                    debug=debug,
                    debug_prn=True,
                )
                for index, company in df.iterrows()
            ]
        )
        await asyncio.sleep(10)
        await index_datasets()

    # end script
    print("🥫 done")
    script_end = time.time()
    print(
        f"Took {script_end - script_start} seconds to consolidate data from {len(config_company_ls.index)} instances"
    )

    #     except Exception as e:
    #         print(e)

    #     finally:

    await asyncio.sleep(10)
    # return logger

# 🚀 Execute Main 🚀


In [ ]:
logger = await main(debug=False, print_progress=False)

starting 0
searching datacenter for playstation
searching datacenter for playstation-ap
searching datacenter for playstation-beta
searching datacenter for playstation-client-engineering
searching datacenter for playstation-core
get_instance_grants: 80 grants returned from playstation-beta
get_instance_grants: 80 grants returned from playstation
get_instance_grants: 80 grants returned from playstation-ap
get_instance_grants: 80 grants returned from playstation-client-engineering
get_instance_grants: 80 grants returned from playstation-core
🎬 searching datacenter for playstation
🎬 searching datacenter for playstation-ap
🎬 searching datacenter for playstation-beta
🎬 searching datacenter for playstation-client-engineering
🎬 searching datacenter for playstation-core
🎉 get_instance_datasets: 3 dataset returned from playstation-core
🆘 get_instance_datasets: error retrieving datasets from playstation-core
🎉 get_instance_datasets: 48 dataset returned from playstation-ap
🆘 get_instance_datasets:

Not supported query type: {"reportDescription": {
    "source": "realtime",
    "dateFrom":"60 minutes ago",
    "dateTo":"now",
    "dateGranularity":"minute:1",
    "reportSuiteID": "sneglobalstore-prod",
    "metrics": [
        {"id": "instances"}],
    "elements": [
        {"id":"page",
         "top":50}]
    }
}


ALERT: unable to parse table
🎉🎉 get_instance_datasets: retrieved 120 from playstation -- starting augmentation
🎉🎉 get_instance_datasets: retrieved 368 from playstation-beta -- starting augmentation
'🎬 upload_with_data: starting retrieval 2022-08-01, 2022-12-31, playstation
'🎬 upload_with_data: starting retrieval 2022-08-01, 2022-12-31, playstation-ap
'🎬 upload_with_data: starting retrieval 2022-08-01, 2022-12-31, playstation-beta
'🎬 upload_with_data: starting retrieval 2022-08-01, 2022-12-31, playstation-client-engineering
'🎬 upload_with_data: starting retrieval 2022-11-18, 2022-12-31, playstation-core


Unclosed connection
client_connection: Connection<ConnectionKey(host='playstation-beta.domo.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='playstation-beta.domo.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='playstation-beta.domo.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='playstation-beta.domo.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='playstation-beta.domo.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='

starting playstation - get_instance_sandbox
upload_data : unexpected error: get_repositories() takes 2 positional arguments but 3 were given
starting playstation-ap - get_instance_sandbox
upload_data : unexpected error: get_repositories() takes 2 positional arguments but 3 were given
starting playstation-beta - get_instance_sandbox
upload_data : unexpected error: get_repositories() takes 2 positional arguments but 3 were given
starting playstation-client-engineering - get_instance_sandbox
upload_data : unexpected error: get_repositories() takes 2 positional arguments but 3 were given
starting playstation-core - get_instance_sandbox
upload_data : unexpected error: get_repositories() takes 2 positional arguments but 3 were given
🥫🥫 indexing datasets
🥫🥫 indexing done
starting 1
searching datacenter for playstation-corporate-planning
searching datacenter for playstation-d2c
searching datacenter for playstation-data-ocean
searching datacenter for playstation-dso-adobe
searching datacenter f

...

ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table


Not supported query type: {"reportDescription": {
    "reportSuiteID":"sneglobalstore-prod",
    "dateFrom": $Yesterday-59,
    "dateTo": $Yesterday,
    "dateGranularity": "Day",
    "metrics": [{"id": "orders"}],
    "elements": [{"id": "product",
                  "classification": "Product Name",
                 "top":500}],
    "segments": [{"id": "s300000566_614a65350d59f81b3f41e832"},
                {"element": "evar2",
                  "classification": "Region",
                  "selected": ["SCEA"]}]
       }
    }
Not supported query type: {"reportDescription": {
    "reportSuiteID":"snepsappglobal",
    "dateFrom": $Yesterday-59,
    "dateTo": $Yesterday,
    "dateGranularity": "Day",
    "metrics": [{"id":"visits"}],
    "elements": [{"id": "page",
                 "search": {"type":"OR", "keywords": ["mobile:psapp:game hub:pre:"]},
                 "top":500}],
    "segments": [{"element":"evar2",
                  "classification": "Region", 
                  "selec

ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table


Not supported query type: {"reportDescription": {
    "reportSuiteID":"snepdrglobal",
    "dateFrom": '2022-10-06',
    "dateTo": $Today,
    "elements": [{"id": "evar40"},
                 {"id": "page",
                  "top": 500},
                 {"id": "evar2",
                  "classification": "Country",
                  "top": 50}],
    "metrics": [{"id": "pageviews"},
                {"id": "visits"},
                {"id": "uniquevisitors"}],
    "segments": [{"id": "s300000566_633636f5d6997f63052180d9"}]
       }
    }
Not supported query type: {"reportDescription": {
    "reportSuiteID":"sneglobalstore-prod",
    "dateFrom": $Yesterday-59,
    "dateTo": $Yesterday,
    "dateGranularity": "Day",
    "metrics": [{"id": "visits"}],
    "elements": [{"id": "product",
                  "classification": "Product Name",
                 "top":500}],
    "segments": [{"id": "s300000566_614a64fdfef1ab11fdcf8fd5"},
                {"element": "evar2",
                  "classifi

ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
ALERT: unable to parse table
🎉🎉 get_instance_datasets: retrieved 550 from playstation-test-platformanalytics -- starting augmentation
'🎬 upload_with_data: starting retrieval 2022-08-01, 2022-12-31, playstation-test-platformanalytics
'🎬 upload_with_data: starting retrieval 2022-08-01, 2022-12-31, playstation-training
'🎬 upload_with_data: starting retrieval 2022-08-15, 2022-12-31, playstation-training-sandbox
starting playstation-test-platformanalytics - get_instance_sandbox
upload_data : unexpected error: get_repositories() takes 2 positional arguments but 3 were given
starting playstation-training - get_instance_sandbox
upload_data : unexpected error: get_repositories() takes 2 positional arguments bu

In [ ]:
await index_datasets()

🥫🥫 indexing datasets
🥫🥫 indexing done


# 🚀 Execute Logging 🚀


In [ ]:
def get_intake():
    import domojupyter as dj

    intake_df = dj.read_dataframe(
        "76b804b4-5e7a-448f-9b04-2dcd2aa52897", query="SELECT * FROM table"
    )
    intake_df.drop(columns=["Timestamp"], inplace=True)
    intake_df.rename(
        columns={
            "Who / What is this instance for?": "use_case",
            "Responsible party(ies)": "responsible_party",
            "Domo Instance": "instance",
            "Jira Instance Label": "jira_label",
        },
        inplace=True,
    )

    return intake_df


intake_df = get_intake()
print(intake_df)

                                             use_case  \
0   Central DOMO Instance for all governed, certif...   
1   Super-secret fight club style instance for Ana...   
2   Audience Platform embedded analytics provider....   
3                        Jae and Oleksii's playground   
4   Japan based Client Engineering team instance, ...   
5   DSO Tokyo owned instance for Corporate Plannin...   
6   Direct to Customer sales team instance.  Team ...   
7                     Users attempting to ingest data   
8   Analytics Engineering Adobe Clearing House of ...   
9   DSO Tokyo instance for their own data explorat...   
10  New loyalty programme - PS Stars - all data re...   
11  Approved instance to channel all mobile data r...   
12  Operations team partner instance (InfoTeam) - ...   
13  Original Analytics Engineering testing instanc...   
14  Audience Platform embedded analytics provider....   
15  Original DSO training instance that was used f...   
16  Global SIE trainign instanc

In [ ]:
import datetime as dt


async def upload_config(intake_df):
    import domojupyter as dj

    report_date = dt.date.today()

    logger = await asyncio.gather(
        *[
            get_config(
                company.instance_auth,
                company.instance_created_date,
                report_date=report_date,
            )
            for index, company in config_company_ls[
                config_company_ls["is_valid"] == 1
            ].iterrows()
        ]
    )

    upload_df = pd.DataFrame(logger)

    upload_df = upload_df.merge(
        intake_df, how="left", left_on="instance", right_on="instance"
    )

    print(upload_df.iloc[0])

    dj.write_dataframe(pd.DataFrame(upload_df), env.CONSOL_INSTANCECONFIG)
    print("🥫 done")


await upload_config(intake_df)

instance                                                           playstation
creation_date                                                       2020-05-06
url                                                       playstation.domo.com
role_admin_membership        [JaeMyong.Wilson@sony.com, john.taft@sony.com,...
role_sie_admin_membership                                                   []
roles                        [sie_instance_admin, admin_reset_account, sie_...
report_date                                                         2023-01-13
groups                       [BDO - Restricted, Content, Default, PXG05 Acc...
domains                                                   [sony.com, domo.com]
allowlist                    [10.84.32.0/20, 10.85.96.0/20, 103.123.136.0/2...
use_case                     Central DOMO Instance for all governed, certif...
responsible_party                                           john.taft@sony.com
jira_label                                          